# Загрузка данных

Для парсинга были взяты следуюшие паблики, которые выпадли первыми при запросе "самые большие паблики вк с анекдотами":
1. [Анекдоты категории б](vk.com/anekdotikategoriib)
2. [Смешные анекдоты](vk.com/smeshnye__anekdoty )
3. [анекдоты категории Б: избранное](vk.com/baneksbest)
4. [• Анекдоты](vk.com/anecdotess)
5. [Анекдоты](vk.com/megaotriv)
6. [Анекдоты](vk.com/anecdotes)
7. [анекдотов.net](vk.com/anekdot)
8. [анекдотов.net16+](vk.com/anekdot18p)
9. [Самые Лучшие Анекдоты ツ](vk.com/best__jokes)
10. [Анекдоты из жизни от анекдотов.net](vk.com/a_story)  

После подгрузки данных была произведена минимальная фильтрация данных:
- Попытка удаления рекламных постов
- Удаление строк маленькой длины (предположительно, подписей к картинкам)

После загрузки данных были применены следующие фильтры:
- Удаление малых постов, которые, вероятнее всего, были подписями к картинкам
- Удаление рекламных постов (попытка)

Итог:

Датасет включает анекдоты разной длины, но с довольно токсичным содержанием.

In [ ]:
import requests
from tqdm import tqdm
import pandas as pd
import time
import re
from typing import List

In [11]:
TOKEN_USER = '...VERSION = 5.131

In [12]:
def parse_public(domain: int, blocks: int, offset: int = 0) -> List[str]:
    texts = []
    for i in tqdm(range(blocks)):
        response = requests.get('https://api.vk.com/method/wall.get',
        params={'access_token': TOKEN_USER,
            'v': VERSION,
            'domain': domain,
            'count': 100,
            'offset': i*100 + offset})
        try:
            data = response.json()['response']['items']
            texts.extend([item['text'].strip() for item in data if item['text'].strip()])
        except Exception as er:
            print(f'Error: {er}')
    return texts

In [13]:
def save_dataframe(df: pd.DataFrame, name: str) -> None:
    df.to_csv(name)

In [21]:
def save_parsed_texts(domains: List[str], blocks: List[int], offsets: List[int], filenames: List[str]) -> None:
    assert len(domains) == len(blocks) == len(offsets) == len(filenames)
    for domain, block, offset, filename in zip(domains, blocks, offsets, filenames):
        texts = parse_public(domain, block, offset)
        df = pd.DataFrame({'text': texts})
        display(df)
        df.to_csv(filename)

In [19]:
domain_list = ['anekdotikategoriib', 'smeshnye__anekdoty', 'baneksbest', 'anecdotess', 'megaotriv', 'anecdotes', 'anekdot',
          'anekdot18p', 'best__jokes', 'a_story']
blocks_list = [250, 678, 18, 980, 1629, 693, 918, 615, 2040, 740]
offset_list = [1, 1, 1, 1, 7, 0, 1, 1, 1, 1]
filename_list = ['categoriiB.csv', 'smeshnye_anekdoty.csv', 'baneksbest.csv', 'anecdotess.csv',
            'megaotriv.csv', 'anecdotes.csv', 'anekdot.csv', 'anekdot18p.csv',
            'best_jokes.csv', 'a_story.csv']

In [22]:
save_parsed_texts(domain_list, blocks_list, offset_list, filename_list)

100%|██████████| 250/250 [04:30<00:00,  1.08s/it]


,text
0,"Я работаю на атомной станции, чтобы прокормить..."
1,"Мужик спрашивает у араба: \n— Слушай, вот ника..."
2,Выпивают три мужика. \nОдин говорит: \n— Если ...
3,Гаишник случайно махнул рукой и останавлилась ...
4,"Идут по лесу Дюймовочка, Белоснежка и 2022 год..."
...,...
21779,"Мужик с кейсом садится в самолёт, после взлёта..."
21780,"Едут в купе грузин, бабка Светлана и украинец...."
21781,"- Блин, мама, я так устала, сегодня четыре пар..."
21782,Женщина звонит в милицию и говорит помогите ме...


100%|██████████| 678/678 [13:19<00:00,  1.18s/it]


,text
0,Сидит рыбак зимой рыбу ловит. Мороз.\nПроходит...
1,— Ты заболел? Когда пойдешь к врачу? \n— Потом...
2,Меня зовут Николай. Уже более 3-х лет я делаю ...
3,"Кашу в голове иногда нужно помешивать, чтобы н..."
4,Мы нарисуем твою любимую на холсте. Ты нам дае...
...,...
25158,"Наша Таня громко плачет, Громко ржет и снова п..."
25159,Единство духа отдавало перегаром.
25160,Хуже дурака только дурак с инициативой…
25161,----------------------------------------------...


100%|██████████| 18/18 [00:23<00:00,  1.30s/it]


...

,text
0,Приезжает натуралист из длительной командировк...
1,Молодые парни смеются в бане над сморщенным чл...
2,Забрал кота. Оставлял у знакомой. Забираю отож...
3,Зaxвaтил Чaпaeв (BИ) co cвoeй дивизиeй ж/д cтa...
4,"Идёт мужик пo лecу, вдpуг видит - избушкa нa к..."
...,...
59507,— Ох них@ясе джунгли п@здец. У тебя тут обезья...
59508,« Физтех» отозвал 2000 своих выпускников. Прич...
59509,"Бабушка поняла, что компот забродил, когда дед..."
59510,Идет колхозное собрание. Отчитывается главный ...


100%|██████████| 2040/2040 [33:11<00:00,  1.02it/s]


,text
0,"- Ваша Честь, я бы хотел жениться!\n- Очень хо..."
1,Пока муж звонит в дверь жена прячет любовника ...
2,"— Почему Ваша дочь диктанты пишет с ошибками, ..."
3,Зайцы все время прыгают и живут всего 8 лет.\n...
4,"Первый ребенок — все стираем, гладим, кипятим!..."
...,...
174202,В постели я - звезда! Руки-ноги раскину и сплю...
174203,"Учительницa - Вовочке: \n- Вовочкa, почему у т..."
174204,"Если лежать вниз головой и плакать,то можно по..."
174205,"Шоколад специально заворачивают в фольгу, чтоб..."


100%|██████████| 740/740 [13:50<00:00,  1.12s/it]


,text
0,Случилось с моим американским другом несколько...
1,"Когда мне было 16 лет, родители отправили меня..."
2,"Неповторимый подарок, который оставит след в п..."
3,"Помню, как-то спустились с шефом в Москве в ме..."
4,История эта произошла в середине прошлого века...
...,...
73088,В нашей коммуналке поселилась чеченка Зарема. ...
73089,"Наш «опытно-экспериментальный стенд», а иначе ..."
73090,У меня друг умеет закатывать глаза: получается...
73091,Известный американский полярный путешественник...


In [14]:
texts = parse_public('anekdotikategoriib', 250, 1)
df = pd.DataFrame({'text': texts})
display(df)
df.to_csv('categoriiB.csv')

100%|██████████| 250/250 [04:44<00:00,  1.14s/it]


,text
0,"Я работаю на атомной станции, чтобы прокормить..."
1,"Мужик спрашивает у араба: \n— Слушай, вот ника..."
2,Выпивают три мужика. \nОдин говорит: \n— Если ...
3,Гаишник случайно махнул рукой и останавлилась ...
4,"Идут по лесу Дюймовочка, Белоснежка и 2022 год..."
...,...
21780,"- Блин, мама, я так устала, сегодня четыре пар..."
21781,Женщина звонит в милицию и говорит помогите ме...
21782,"""Zu Ficken"",-сказал Штирлиц, не найдя документ..."
21783,Все думают что мужчины не стонут во время секс...


In [17]:
texts = parse_public('smeshnye__anekdoty', 678, 1)
df = pd.DataFrame({'text': texts})
df.to_csv('smeshnye_anekdoty.csv')

 23%|██▎       | 154/678 [03:14<11:01,  1.26s/it]


KeyboardInterrupt: 

In [ ]:
all_files = ['categoriiB.csv', 'smeshnye_anekdoty.csv', 'baneksbest.csv', 'anecdotess.csv',
            'megaotriv.csv', 'anecdotes.csv', 'anekdot.csv', 'anekdot18p.csv',
            'best_jokes.csv', 'a_story.csv', 'anek_net.csv', 'an18p.csv',
            'lookpics.csv', 'anekdotshort.csv', 'anekdotpremium.csv', 'luchshiyeanekdoty.csv',
            'jewishpugs.csv', 'lolstory.csv']
len(all_files)

18

In [23]:
bad_words = {
    'http', 'https', 'vk', 'com', 'vkontakte', 'реклама', 'заработать', 'заработок',
    'mail', 'выбор', 'api', 'лайк', 'лайки', 'репост', 'инвестиции', 'инветициях',
    'img', 'jpg', 'source', 'вконтакте'
}

result = []

for file in tqdm(filename_list):
    data = pd.read_csv(file)

    for line in data['text'].astype(str).values:
        line = line.strip()
        if len(line) < 40 or any(bad_word in line.lower() for bad_word in bad_words):
            continue
        result.append(line)


100%|██████████| 10/10 [01:24<00:00,  8.45s/it]


In [24]:
data = pd.DataFrame({'text':result})
display(data)

,text
0,"Я работаю на атомной станции, чтобы прокормить..."
1,"Мужик спрашивает у араба: \n— Слушай, вот ника..."
2,Выпивают три мужика. \nОдин говорит: \n— Если ...
3,Гаишник случайно махнул рукой и останавлилась ...
4,"Идут по лесу Дюймовочка, Белоснежка и 2022 год..."
...,...
600531,В нашей коммуналке поселилась чеченка Зарема. ...
600532,"Наш «опытно-экспериментальный стенд», а иначе ..."
600533,У меня друг умеет закатывать глаза: получается...
600534,Известный американский полярный путешественник...


In [25]:
ENGletters = list('abcdefghijklmnopqrstuvwxyz' + 'abcdefghijklmnopqrstuvwxyz'.upper())

In [26]:
data = data[~data['text'].str.contains('|'.join(ENGletters))]
data.reset_index(drop=True)

,text
0,"Я работаю на атомной станции, чтобы прокормить..."
1,"Мужик спрашивает у араба: \n— Слушай, вот ника..."
2,Выпивают три мужика. \nОдин говорит: \n— Если ...
3,Гаишник случайно махнул рукой и останавлилась ...
4,"Идут по лесу Дюймовочка, Белоснежка и 2022 год..."
...,...
504330,Года четыре назад в своем дворике я услышал эт...
504331,Случилось это во времена моей службы в армии. ...
504332,"Наш «опытно-экспериментальный стенд», а иначе ..."
504333,У меня друг умеет закатывать глаза: получается...


In [27]:
data = data.drop_duplicates()
data = data.reset_index(drop=True)
data

,text
0,"Я работаю на атомной станции, чтобы прокормить..."
1,"Мужик спрашивает у араба: \n— Слушай, вот ника..."
2,Выпивают три мужика. \nОдин говорит: \n— Если ...
3,Гаишник случайно махнул рукой и останавлилась ...
4,"Идут по лесу Дюймовочка, Белоснежка и 2022 год..."
...,...
178899,"Подслушано в метро. Судя по всему, девачко лет..."
178900,"Случилось лет 11 назад, в родном городе Одессе..."
178901,История эта произошла на улице города... впроч...
178902,Случилось это во времена моей службы в армии. ...


In [28]:
unique_ind = data['text'].apply(lambda x: re.sub(r'[^а-яА-Я0-9]', '', x.lower())).drop_duplicates().index.values
unique_ind

array([     0,      1,      2, ..., 178897, 178899, 178901])

In [29]:
data = data.iloc[unique_ind]
data = data.reset_index(drop=True)
data

,text
0,"Я работаю на атомной станции, чтобы прокормить..."
1,"Мужик спрашивает у араба: \n— Слушай, вот ника..."
2,Выпивают три мужика. \nОдин говорит: \n— Если ...
3,Гаишник случайно махнул рукой и останавлилась ...
4,"Идут по лесу Дюймовочка, Белоснежка и 2022 год..."
...,...
117582,"история реальная, сам свидетель. итак, на рабо..."
117583,Я помню мы в институте как-то раз флюорографию...
117584,Квартирный вопрос \n\n— —--------------— \n\nЛ...
117585,"Подслушано в метро. Судя по всему, девачко лет..."


In [31]:
data.to_csv('russian_vk_jokes.csv', index=False)